In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from tavily import TavilyClient
tavilyClient = TavilyClient()

In [13]:
from langchain.tools import tool

@tool()
def search_recipes_by_ingredients(query: str):
  """Searchs recipes by ingredients from WEB"""
  return tavilyClient.search(query)

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

config = {"configurable": {"thread_id": "1"}}

agent = create_agent(
    model='gpt-5-nano',
    system_prompt="You are a cousine chief and helps the user to create recipes by their remanescent ingredints. Before search for recipes, analise if the image contains ingredients to make food. If not, responds with : IMAGE INVALID",
    tools=[search_recipes_by_ingredients],
    checkpointer=InMemorySaver()
)

In [17]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [18]:
import base64 

uploaded_file = uploader.value[0]

content_mv = uploaded_file["content"]

img_bytes = bytes(content_mv)

img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [20]:
from langchain.messages import HumanMessage

multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Search recipes to me based on this ingredients"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

response = agent.invoke(
    {"messages": [multimodal_question]},
    config
)

print(response['messages'][-1].content)

Here are suggests I found that match your ingredients (grapefruit, lemon, orange, kiwi, blueberries, almonds, yogurt, honey, ginger, turmeric). I pulled a mix of direct recipes and ideas you can adapt.

Best matching recipes:
- Kiwi Grapefruit Salad | Kowalski's Markets
  - Why it fits: includes kiwi, grapefruit, plain yogurt, and honey; you can top with blueberries and chopped almonds for crunch.
  - Link: https://www.kowalskis.com/recipes/breakfast-brunch/kiwi-grapefruit-salad?srsltid=AfmBOoolgtERPYvx7ZRROQuffU0VArB9_qLeKQfViCCPTf7UbvxpzqX3
  - Note: Core idea you can customize with your extra ingredients.

- 4-Ingredient Wellness Shots (Grapefruit + Lemon + Ginger + Turmeric)
  - Why it fits: uses citrus and ginger/turmeric; easy to make as a quick wellness drink; you can add a touch of honey.
  - Link: https://www.instagram.com/reel/DHgsqSquJFz/?hl=en

- Ginger & Turmeric Shots (Orange + Lemon + Ginger + Turmeric)
  - Why it fits: citrus + ginger + turmeric in a small, potent forma

In [ ]:
## Invoking again to test memory

response = agent.invoke(
    {"messages": [multimodal_question, HumanMessage(content="Which of these recipes is the healthiest?")]},
    config
)

print(response['messages'][-1].content)

A mais saudável, entre as opções que discutimos, costuma ser a Citrus-Blueberry Yogurt Power Bowl.

Por quê:
- Proteína: o iogurte entrega proteína, ajudando a saciar e manter a massa muscular.
- Fibra e carboidratos complexos: as frutas (kiwi, grapefruit, blueberry, laranja) fornecem fibra e vitaminas sem picos de açúcar.
- Gorduras saudáveis e crocância: as amêndoas trazem gorduras boas e proteína adicional.
- Antioxidantes: as frutas cítricas, kiwi e mirtilos são ricos em vitaminas C e outros antioxidantes.
- Versatilidade de controle de açúcar: é fácil reduzir ou eliminar o mel se você quiser menos açúcar.

Observação rápida sobre as outras opções:
- Kiwi Grapefruit Salad: saudável e refrescante, bom equilíbrio, mas pode ter menos proteína; ótimo para complementar com mais iogurte ou queijo cottage se desejar mais proteína.
- Immunity-Boosting Breakfast Bowl: similar à primeira em base, mas pode ter mais açúcares se usar muito mel; ainda é boa se você ajustar o açúcar.
- Shots de g